In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
spark = SparkSession.builder.appName("sparkdev-kafka-streaming demo") \
    .config("spark.jars", "/opt/homebrew/Cellar/apache-spark/3.3.0/libexec/jars/spark-sql-kafka-0-10_2.13-3.3.2.jar,/opt/homebrew/Cellar/apache-spark/3.3.0/libexec/jars/kafka-clients-3.3.2.jar") \
    .master("local[*]").getOrCreate()

In [6]:
# Set up the Kafka parameters
kafka_broker_url = "localhost:9092"
kafka_topic_name = "orders"

# Define the Kafka schema
kafka_schema = StructType() \
    .add("id", IntegerType()) \
    .add("amount", StringType()) \
    .add("date", IntegerType())

# Create the streaming dataframe to read from Kafka
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker_url) \
    .option("subscribe", kafka_topic_name) \
    .load()

# Convert the value column from binary to string and deserialize the JSON data
deserialized_df = kafka_df \
    .select(from_json(col("value").cast("string"), kafka_schema).alias("data")) \
    .select("data.*")

# Perform transformations on the data
transformed_df = deserialized_df \
    .filter(col("age") > 18) \
    .groupBy("name") \
    .agg(avg("age").alias("average_age"))

# Write the transformed data to a console for testing
query = transformed_df \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

# Wait for the query to terminate
query.awaitTermination()

Py4JJavaError: An error occurred while calling o40.load.
: java.lang.NoClassDefFoundError: scala/$less$colon$less
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateStreamOptions(KafkaSourceProvider.scala:338)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.sourceSchema(KafkaSourceProvider.scala:71)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:236)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:118)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:118)
	at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scala:34)
	at org.apache.spark.sql.streaming.DataStreamReader.loadInternal(DataStreamReader.scala:168)
	at org.apache.spark.sql.streaming.DataStreamReader.load(DataStreamReader.scala:144)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(N